 # Visualisation du modèle ConditionalFlowGenerator2d depuis Hugging Face Hub



 Ce notebook vous montre comment :

 - Télécharger le checkpoint depuis le Hub et charger le modèle.

 - Charger un sous-ensemble du jeu de données.

 - Générer des prédictions avec la méthode `sample_most_probable` (avec 100 échantillons) et visualiser les résultats.




 ## 1. Téléchargement du modèle depuis Hugging Face et visualisation

In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

# Import depuis huggingface_hub pour télécharger le checkpoint
from huggingface_hub import hf_hub_download

# On suppose que votre modèle (la classe ConditionalFlowGenerator2d) est accessible via votre repo local.
# Par exemple, vous pouvez avoir un dossier "models" dans votre repository.
from models import ConditionalFlowGenerator2d
from dataset import load_dataset  # La fonction de chargement de dataset de votre projet
from visu import denormalize_variable, transform_longitude  # Fonctions de visualisation/utilitaires


 ### Téléchargement du checkpoint depuis Hugging Face Hub



 Remplacez `repo_id` par l'identifiant de votre dépôt (par exemple `"votre_nom_utilisateur/mon_modele_cf"`)

 et `filename` par le nom de votre fichier de checkpoint (par exemple `"checkpoint_epoch_10.pth"`).

In [2]:
# Paramètres de téléchargement
repo_id = "pcesar/FlowGAN"  # <-- Remplacez par votre repo Hugging Face
filename = "model_1_16_low_reco.pth"              # <-- Nom de votre checkpoint

# Téléchargement du fichier (il sera sauvegardé dans le cache Hugging Face)
checkpoint_path = hf_hub_download(repo_id=repo_id, filename=filename)
print("Checkpoint téléchargé depuis Hugging Face :", checkpoint_path)


model_1_16_low_reco.pth:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

Checkpoint téléchargé depuis Hugging Face : /home/ensta/ensta-cesar/.cache/huggingface/hub/models--pcesar--FlowGAN/snapshots/eefda1b6521a11d548b96297391a6bf277e898f3/model_1_16_low_reco.pth


 ### Chargement du modèle depuis le checkpoint



 On définit une fonction simple pour charger le checkpoint et instancier le modèle.



 La fonction récupère notamment le nombre de flows (par défaut 4 s'il n'est pas précisé).

In [5]:
def load_checkpoint_cf(checkpoint_path, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    nb_flows = checkpoint.get("nb_flows", 16)
    print("Nombre de flows dans le checkpoint :", nb_flows)
    gen = ConditionalFlowGenerator2d(
        context_channels=7,
        latent_channels=3,
        num_flows=nb_flows
    ).to(device)
    gen.load_state_dict(checkpoint['gen_state_dict'])
    gen.eval()
    return gen

# Sélection de l'appareil (GPU si disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_checkpoint_cf(checkpoint_path, device)
print("Modèle chargé et en mode évaluation.")


Nombre de flows dans le checkpoint : 16
Modèle chargé et en mode évaluation.


/tmp/ipykernel_1494217/155791988.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


 ### Chargement d'une partie du jeu de données pour la visualisation



 Ici, nous utilisons la fonction `load_dataset` de votre projet pour charger le jeu de données.

 Nous allons utiliser le jeu de validation.



 **Note :** adaptez le paramètre `root_dir` à l'emplacement de vos données.

In [ ]:
# Paramètres du dataset (à ajuster selon votre configuration)
dataset_dir = "/home/ensta/ensta-cesar/era_5_data/"  # Modifiez ce chemin si besoin
datasets = load_dataset(
    nb_file=10,
    train_val_split=0.8,
    year0=1979,
    root_dir=dataset_dir,
    normalize=True
)
val_dataset = datasets["val"]

# Utilisation d'un DataLoader pour récupérer un batch
from torch.utils.data import DataLoader
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0)

# Récupérons un premier batch pour tester
batch_data = next(iter(val_loader))
print("Batch récupéré.")


 ### Préparation de l'entrée pour le modèle



 Dans votre pipeline, l'entrée se construit en concaténant les données d'`input`, `masks` et `coords`.

 On remanie ensuite les dimensions pour convenir au modèle.

In [ ]:
inputs = batch_data["input"].to(device)
masks = batch_data["masks"].to(device)
lat_coord = batch_data["coords"][0].unsqueeze(1).to(device)
lon_coord = batch_data["coords"][1].unsqueeze(1).to(device)
coords = torch.cat([lat_coord, lon_coord], dim=1)
x = torch.cat([inputs, masks, coords], dim=1)
# Remise en forme : (batch, channels, width, height)
x = x.permute(0, 3, 2, 1)
print("Forme de l'entrée :", x.shape)


 ### Génération des prédictions avec 100 échantillons



 Ici, nous utilisons la méthode `sample_most_probable` (votre méthode de prédiction) en spécifiant `num_samples=100`.

 Cela vous permet de générer la vidéo finale en considérant 100 échantillons par prédiction.

In [ ]:
with torch.no_grad():
    fake = model.sample_most_probable(x, num_samples=100)

# Réorganisation pour la visualisation : (num_samples, width, height, channels)
fake = fake.permute(0, 3, 2, 1).cpu().numpy()
print("Forme de la prédiction :", fake.shape)


 ### Visualisation des prédictions



 Ici, nous allons visualiser le canal de température (indice 0) d'une des prédictions.



 Si vos données sont normalisées, nous appliquons la dénormalisation à l'aide des paramètres fournis par le dataset.

In [ ]:
# Récupération des paramètres de normalisation depuis le dataset
norm_params = val_dataset.get_norm_params()

# Extraction du canal température (indice 0)
temp_pred = fake[:, :, :, 0]
# Si le dataset est normalisé, on dénormalise et on convertit de Kelvin en Celsius
if val_dataset.normalize:
    temp_pred = denormalize_variable(temp_pred, norm_params['2m_temperature']) - 273.15
else:
    temp_pred = temp_pred - 273.15

# Pour la visualisation, nous transformons les longitudes si nécessaire
temp_pred = transform_longitude(temp_pred)

# Définir la grille géographique à partir des dimensions
nlat = temp_pred.shape[1]
nlon = temp_pred.shape[2]
lat_vals = np.linspace(-90, 90, nlat)
lon_vals = np.linspace(-180, 180, nlon)

# Visualisation d'une prédiction (par exemple, la première des 100)
plt.figure(figsize=(8, 6))
plt.imshow(temp_pred[0], cmap='RdBu_r', origin='lower', extent=[lon_vals.min(), lon_vals.max(), lat_vals.min(), lat_vals.max()])
plt.title("Prédiction de la température (°C)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.colorbar(label="Température (°C)")
plt.show()


 ### Remarques complémentaires



 - Vous pouvez adapter ce notebook pour générer des animations (vidéos) en utilisant vos fonctions de visualisation telles que `compute_animation_for_scalar` si vous souhaitez créer une vidéo finale.

 - Pour générer et sauvegarder une vidéo, pensez à définir un répertoire de sortie et à appeler la fonction en passant les données et paramètres (fps, année, etc.).



 Ce notebook constitue une base pour tester votre modèle chargé depuis Hugging Face et visualiser ses prédictions sur un sous-ensemble du jeu de données.